Get events from category codes.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import zscore
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
backup_df = pd.read_csv('../data/local/raw/backup_topics_and_ids.csv')
display(backup_df)

,title,clean_title,original_title,genres,director,release_year,runtime,budget,revenue,popularity,tmdb_rating,tmdb_votes,imdb_rating,imdb_votes,language,tmdb_id,imdb_id,topics,doesthedog_id
0,Bad Boys for Life,bad boys for life,Bad Boys for Life,"thriller, action, crime","Bilall Fallah, Adil El Arbi",2020,124,90000000,426505244,67.7,7.1,8251,6.5,186716,English,38700,tt1502397,"167,177,180,184,188,192,193,198,208,211,228,23...",22448.0
1,Avatar: The Way of Water,avatar the way of water,Avatar: The Way of Water,"science fiction, adventure, action",James Cameron,2022,192,460000000,2320250281,158.5,7.6,11946,7.5,518001,English,76600,tt1630029,"158,161,164,167,168,177,181,188,189,190,191,19...",983966.0
2,Pet Sematary,pet sematary,Pet Sematary,"horror, thriller","Dennis Widmyer, Kevin Kölsch",2019,100,21000000,113118226,26.1,5.7,3186,5.7,100618,English,157433,tt0837563,"153,158,161,164,165,168,171,176,177,180,184,18...",10857.0
3,How to Train Your Dragon: The Hidden World,how to train your dragon the hidden world,How to Train Your Dragon: The Hidden World,"animation, family, adventure",Dean DeBlois,2019,104,129000000,524580592,207.5,7.8,6381,7.4,158269,English,166428,tt2386490,"190,196,211,222,229,233,243,244,252,270,289,35...",17531.0
4,Star Wars: The Rise of Skywalker,star wars the rise of skywalker,Star Wars: The Rise of Skywalker,"adventure, action, science fiction",J.J. Abrams,2019,142,416000000,1074144248,63.2,6.3,9886,6.4,507321,English,181812,tt2527338,"161,164,167,168,181,184,188,198,202,207,214,23...",22008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,The Last Breath,the last breath,The Last Breath,"horror, thriller",Joachim Hedén,2024,96,30000,273650,48.9,6.5,79,4.7,1563,English,1241674,tt14827150,NaN,18937.0
1163,Moana 2,moana 2,Moana 2,"animation, adventure, family, comedy","David G. Derrick Jr., Dana Ledoux Miller, Jaso...",2024,100,150000000,600055655,4485.0,6.8,424,7.1,30852,English,1241982,tt13622970,"167,207,213,274,298,311,328,335,337,339,346,35...",1151858.0
1164,Sound of Hope: The Story of Possum Trot,sound of hope the story of possum trot,Sound of Hope: The Story of Possum Trot,drama,Joshua Weigel,2024,130,8500000,11721425,58.5,6.7,23,7.1,1782,English,1242526,tt6881490,"212,218,294",1152270.0
1165,Waves,waves,Vlny,"thriller, drama, history",Jiří Mádl,2024,131,3400000,6365214,6.0,8.3,11,8.5,866,Czech,1251621,tt27587485,"158,167,168,176,180,181,188,192,193,197,201,20...",21753.0


Add dictionary for event description

In [ ]:
event_dict = {
    '153': 'dogs dying',
    '319': 'animals harmed during making',
    '189': 'animals (besides dog/cat/horse) dying',
    '229': 'animal abuse',
    '231': 'dog fighting',
    '252': 'dead animals',
    '355': 'sad animals',
    '338': 'harmed rabbits',
    '186': 'cats dying',
    '285': 'pets die',
    '196': 'dragons dying',
    '165': 'spiders',
    '332': 'alligators/crocodiles',
    '214': 'snakes',
    '337': 'sharks',
    '213': 'bugs',
    '183': 'horses dying',
    '293': 'child abandonment',
    '350': 'abandoned animals',
    '363': 'leave without goodbye',
    '242': 'stalking',
    '347': 'abused forgives abuser',
    '304': 'abused becomes abuser',
    '237': 'gaslighting',
    '365': 'abuse with belt',
    '218': 'child abuse',
    '367': 'women brutalized for spectacle',
    '219': 'domestic violence',
    '330': 'abusive parents',
    '225': 'alcohol abuse',
    '230': 'addiction',
    '193': 'drug use',
    '320': 'pedophilia',
    '308': 'held under water',
    '326': 'rape mentions',
    '321': 'beaten up by bully',
    '274': 'restraints',
    '182': 'sexual assault',
    '360': 'mouth covering',
    '315': 'sexual assault on men is a joke',
    '292': 'onscreen rape',
    '342': 'women slapped',
    '299': 'druggings',
    '352': 'hand damage',
    '362': 'dislocations',
    '361': 'throat mutilation',
    '245': 'choking',
    '331': 'decapitation',
    '254': 'cannibalism',
    '282': 'crushed to death',
    '309': 'chokings',
    '164': 'people being burned alive',
    '240': 'buried alive',
    '296': 'body horror',
    '250': 'amputation',
    '223': 'heads getting squashed',
    '280': 'Achilles Tendon injury',
    '177': 'shaving or cutting',
    '281': 'asphyxiation',
    '258': 'genital trauma/mutilation',
    '298': 'unconscious',
    '216': 'bones breaking',
    '206': 'seizures',
    '210': 'teeth damage',
    '171': 'finger or toe mutilation',
    '203': 'torture',
    '271': 'falling down stairs',
    '211': 'falling deaths',
    '200': 'eye mutilation',
    '343': 'stabbings',
    '267': 'excessive gore',
    '227': 'hangings',
    '284': 'infant abduction',
    '287': 'sexualized minor',
    '158': 'kids dying',
    '357': 'bedbugs',
    '289': 'self-sacrifice',
    '305': 'non-human death',
    '311': 'someone dies',
    '328': 'major character dies',
    '353': 'disabled played by able-bodied',
    '272': 'R-slur',
    '275': 'overdose',
    '168': 'parents dying',
    '253': 'dear toy destruction',
    '243': 'kidnapping',
    '313': 'family dies',
    '241': 'cheating',
    '207': 'ghosts',
    '161': 'jump scares',
    '176': 'shower scenes',
    '335': 'bodies of water',
    '174': 'clowns',
    '224': 'demonic possession',
    '312': 'trypophobia',
    '297': 'razors',
    '295': 'mannequins',
    '201': 'vomiting',
    '257': 'wet/soiled pants',
    '324': 'farting',
    '322': 'someone is eaten',
    '354': 'pooping',
    '180': 'spitting',
    '255': 'audio gore',
    '283': '9/11',
    '256': 'copaganda',
    '345': 'incarceration',
    '359': 'trans person depicted predatorily',
    '310': 'deadnaming / birthnaming',
    '368': 'LGBT+ person outed',
    '314': 'transphobic slurs',
    '301': 'bisexual cheating',
    '329': 'artifact destroyed',
    '190': 'needles or syringes are used',
    '205': 'electro-therapy',
    '192': 'hospital scenes',
    '358': 'menstruation',
    '204': 'cancer',
    '220': 'mental institutions',
    '199': 'self harming',
    '349': 'autism misrepresented',
    '263': 'violent mentally ill person',
    '370': 'dissociation, depersonalization, or derealization',
    '302': 'D.I.D. misrepresentation',
    '248': 'autism abuse',
    '286': 'suicide attempts',
    '334': 'reality unhinged',
    '336': 'mental illness',
    '260': 'misophonia',
    '235': 'anxiety attacks',
    '323': 'ABA therapy',
    '259': '"I\'ll kill myself"',
    '306': 'body dysphoria',
    '195': 'body dysmorphia',
    '202': 'claustrophobic scenes',
    '217': 'eating disorders',
    '187': 'people dying by suicide',
    '265': 'PTSD',
    '348': 'meltdowns',
    '356': 'underwater scenes',
    '339': 'sudden loud noises',
    '181': 'shaky cam',
    '261': 'crying babies',
    '366': 'screaming',
    '290': 'obscene language/gestures',
    '167': 'flashing lights or images',
    '318': 'being watched',
    '268': 'fourth wall',
    '264': 'stillbirths',
    '215': 'miscarriages',
    '266': 'babies/unborn',
    '228': 'childbirth',
    '238': 'abortions',
    '239': 'pregnant people deaths',
    '247': 'homophobic slurs',
    '244': 'ableism',
    '246': 'antisemitism',
    '262': 'man in a dress jokes',
    '226': 'misgendering',
    '194': 'LGBT people dying',
    '233': 'fat jokes',
    '303': 'aphobia',
    '251': 'n-word usage',
    '234': 'black guy dies first',
    '212': 'hate speech',
    '294': 'minority misrepresentation',
    '325': 'blackface',
    '277': 'large age gap',
    '351': 'religion discussed',
    '369': 'demons or Hell',
    '279': 'nude scenes',
    '307': 'bestiality',
    '317': 'lose virginity',
    '276': 'sexual objectification',
    '236': 'incest',
    '197': 'sexual content',
    '364': 'BDSM',
    '273': 'male crying ridicule',
    '278': 'strokes',
    '288': 'chronic illnesses',
    '291': 'dementia/Alzheimer\'s',
    '327': 'terminal illness',
    '300': 'fat suits',
    '270': 'homelessness',
    '341': 'existentialism',
    '316': 'anti-abortion',
    '209': 'Santa (et al) spoilers',
    '346': 'end credit scenes',
    '222': 'sad endings',
    '269': 'car honk / tire screech',
    '184': 'car crashes',
    '198': 'planes crashing',
    '208': 'people getting hit by cars',
    '191': 'drownings',
    '188': 'blood or gore',
    '221': 'nuclear explosions',
    '232': 'gun violence'
}
